### Example creating the stiffness for a linear elasticity element using tensors or matrices

In [1]:
using JuAFEM

In [2]:
E = 200e9
ν = 0.3
λ = E*ν / ((1+ν) * (1 - 2ν))
μ = E / (2(1+ν))
δ(i,j) = i == j ? 1.0 : 0.0
g(i,j,k,l) = λ*δ(i,j)*δ(k,l) + μ*(δ(i,k)*δ(j,l) + δ(i,l)*δ(j,k))

# Create a random symmetric material stiffness
C = rand(SymmetricTensor{4, 2})

Ee = [C[1,1,1,1] C[1,1,2,2] C[1,1,1,2];
      C[2,2,1,1] C[2,2,2,2] C[2,2,1,2];
      C[1,2,1,1] C[1,2,2,2] C[1,2,1,2]];

In [3]:
function_space = Lagrange{2, RefCube, 1}()
quad_rule = QuadratureRule{2, RefCube}(2)
fe_values = FECellScalarValues(quad_rule, function_space);
fe_vector_values = FECellVectorValues(quad_rule, function_space);

x = [0. 1 1 0;
     0 0 1 1]
x_vec = reinterpret(Vec{2, Float64}, x, (4,));

## Stiffness

In [4]:
function ke_element_mat!{T, dim}(Ke, X::Vector{Vec{dim, T}}, fe_values::FECellScalarValues{dim}, Ee, B, DB, BDB)
    n_basefuncs = getnbasefunctions(fe_values)
    @assert length(X) == n_basefuncs
    
    reinit!(fe_values, X)
    for q_point in 1:getnquadpoints(fe_values)
        for i in 1:n_basefuncs
            dNdx = shape_gradient(fe_values, q_point, i)[1]
            dNdy = shape_gradient(fe_values, q_point, i)[2]
            B[1, 2*i - 1] = dNdx
            B[2, 2*i - 0] = dNdy
            B[3, 2*i - 0] = dNdx
            B[3, 2*i - 1] = dNdy
        end
        
        A_mul_B!(DB, Ee, B)
        At_mul_B!(BDB, B, DB)
        scale!(BDB, getdetJdV(fe_values, q_point))
        for p in 1:size(Ke,1)
            for q in 1:size(Ke,2)
                Ke[p, q] += BDB[p, q]
            end
        end
    end
    
    return Ke
end

ke_element_mat! (generic function with 1 method)

In [5]:
function ke_element!{T,dim}(Ke, X::Vector{Vec{2, T}}, fe_values::FECellScalarValues{dim}, C2)
    n_basefuncs = getnbasefunctions(fe_values)
    @assert length(X) == n_basefuncs
    reinit!(fe_values, X)
    @inbounds for q_point in 1:getnquadpoints(fe_values)
        for a in 1:n_basefuncs
            for b in 1:n_basefuncs
                ∇ϕa = shape_gradient(fe_values, q_point, a)
                ∇ϕb = shape_gradient(fe_values, q_point, b)
                # TODO: This would be faster as ∇ϕa ⋅ C ⋅ ∇ϕb
                Ke_e = C2 ⊡ (∇ϕb ⊗ ∇ϕa) * getdetJdV(fe_values, q_point)
                for d1 in 1:dim, d2 in 1:dim
                    Ke[dim*(a-1) + d1, dim*(b-1) + d2] += Ke_e[d1,d2]
                end
            end
        end
    end
    return Ke
end

ke_element! (generic function with 1 method)

In [6]:
function ke_element2!{T,dim}(Ke, X::Vector{Vec{2, T}}, fe_values::FECellVectorValues{dim}, C)
    n_basefuncs = getnbasefunctions(fe_values)
    @assert length(X) * dim == n_basefuncs
    reinit!(fe_values, X)
    ɛ = [zero(SymmetricTensor{2, dim, T}) for i in 1:n_basefuncs]
    @inbounds for q_point in 1:getnquadpoints(fe_values)
        for i in 1:n_basefuncs
            ɛ[i] = symmetric(shape_gradient(fe_values, q_point, i)) 
        end
        dΩ = getdetJdV(fe_values, q_point)
        for i in 1:n_basefuncs
            for j in 1:n_basefuncs
                Ke[i, j] += (ɛ[i] ⊡ C ⊡ ɛ[j]) * dΩ
            end
        end
    end
    return Ke
end

ke_element2! (generic function with 1 method)

In [11]:
x = [0. 1 1 0;
     0 0 1 1]
x_vec = reinterpret(Vec{2, Float64}, x, (4,))
Ke2 = zeros(8,8)
# Note the transpose
C2 = Tensor{4, 2}((i,j,k,l) -> C[i,l,k,j])
@time ke_element!(Ke2, x_vec, fe_values, C2)
Ke3 = zeros(8,8)
@time ke_element2!(Ke3, x_vec, fe_vector_values, C);

  0.000006 seconds (4 allocations: 160 bytes)
  0.000009 seconds (6 allocations: 512 bytes)


In [12]:
B = zeros(3, 8)
DB = zeros(3,8)
BDB = zeros(8,8)
Ke = zeros(8,8)
fill!(Ke, 0.0)
@time ke_element_mat!(Ke, x_vec, fe_values, Ee, B, DB, BDB);

  0.000033 seconds (4 allocations: 160 bytes)


In [13]:
norm(Ke - Ke2) / norm(Ke)

1.2732549793499391e-16

In [14]:
norm(Ke - Ke3) / norm(Ke)

7.695463072134969e-17